## MIS780 - Advanced Artificial Intelligence for Business

## Week 5 - Part 1: Classifying Images with CNN

In this session, you will classify Images with deep learning packages (Tensorflow and Keras).


## Table of Content
   
1. [Platform Preperation](#cell_Preperation)     

    
2. [Data Preperation](#cell_DataPreperation)


3. [Deep Learning Model Construction](#cell_Model)


4. [Model Execution](#cell_ModelExecution)


  

<a id = "cell_Preperation"></a>
### 1. Platform Preperation

Make sure to change Notebook Settings using GPU as hardware accelerator in Google Colab.

*Markdown formats: All tables and images will be aligned to the left.*

In [ ]:
%%html
<style>table {float:left}</style>
<style>img {float:left}</style>

Load necessary libraries

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Check GPUs on this machine

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

Create a function to plot images.

In [ ]:
def plot_images(ims, figsize=(12,12), cols=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        if (ims.shape[-1] != 3):
            ims = ims = ims[:,:,:,0]
    f = plt.figure(figsize=figsize)
    rows=len(ims)//cols if len(ims) % cols == 0 else len(ims)//cols + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

Create a function to plot history.

In [ ]:
def plot_hist(h, xsize=6, ysize=5):
    # Prepare plotting
    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]

    # Get training and validation keys
    ks = list(h.keys())
    n2 = math.floor(len(ks)/2)
    train_keys = ks[0:n2]
    valid_keys = ks[n2:2*n2]

    # summarize history for different metrics
    for i in range(n2):
        plt.plot(h[train_keys[i]])
        plt.plot(h[valid_keys[i]])
        plt.title('Training vs Validation '+train_keys[i])
        plt.ylabel(train_keys[i])
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.draw()
        plt.show()

    return

<a id = "cell_DataPreperation"></a>
### 2. Data Preperation

The below codes provide the procedures to load **CIFAR10** data sets. The data set consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.  

In [ ]:
from tensorflow.keras.datasets import cifar10

# Data parameters
img_rows, img_cols = 32, 32
channels = 3

num_classes = 10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

**Prepare the selected data set and split into training and testing sets.**

In [ ]:
from tensorflow.keras.utils import to_categorical

# Reshape images for processing
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)

# Convert pipxel values to 8 bits
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print('Train shape: x=', x_train.shape, ', y=', y_train.shape)
print('Test shape: x=', x_test.shape, ', y=', y_test.shape)

Show a sample of data

In [ ]:
plot_images(x_train[0:20], cols=5,figsize=[7,7])

<a id = "cell_Model"></a>
### 3. Deep Learning Model Construction

Load the required libraries for CNN construction  

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D, Activation, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, Callback, EarlyStopping
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Nadam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import regularizers

Build some models with varied architectures (e.g. layers, nodes)

In [ ]:
#Traditional ANN with one Flatten layer, one Hidden Layer, and one Output layer
#All layers are fully connected following traditional ANN architecture.
def model_1():
    model = Sequential()
    model.add(Flatten(input_shape=(img_rows, img_cols, channels)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    return model

In [ ]:
#CNN model with two Convolution layers, one Pooling layer with max pooling,
#which are stacked on top of a traditional ANN model (with the same architecture as the model 1)
def model_2():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(img_rows, img_cols, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

Define callback to record training performnace.

In [ ]:
# Keras callbacks (when Tensorboard installed)
keras_callbacks = [EarlyStopping(monitor='val_loss', patience=20, verbose=0)]


<a id = "cell_ModelExecution"></a>
### 4. Model Execution

We select one of the above models to carry out the experiment.

In [ ]:
model = model_1()

Compile and fit the model using `RMSprop` optimizer. <br/>
*See optimisers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/*

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=RMSprop(learning_rate=0.001,weight_decay=1e-6),
              metrics='accuracy')

#Other Optimizers that can be used instead of RMSprop
#Adadelta(lr=0.001, rho=0.95, epsilon=1e-07)
#Adadelta(lr=0.05, rho=0.99, epsilon=1e-07)
#Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

hist = model.fit(x_train, y_train,
      batch_size=128,
      epochs=100,
      verbose=2,
      validation_data=(x_test, y_test),
      validation_split=0.2,
      callbacks=keras_callbacks)

Evaluate the model

In [ ]:
# Evaluate on training data
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', round(train_score[0], 4))
print('Train accuracy:', round(train_score[1], 4), '\n')

# Evaluate on test data
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', round(test_score[0], 4))
print('Test accuracy:', round(test_score[1], 4))

Plot history of learning

In [ ]:
plot_hist(pd.DataFrame(hist.history))

Computing the **accuracy, precision, recall, f1-score, and support**

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

# Make predictions on the test set
y_pred = model.predict(x_test)

# Convert the predicted labels to continuous-multioutput format
y_pred_continuous = np.round(y_pred)

# Convert the predicted labels to multiclass format
y_pred_multiclass = np.argmax(y_pred, axis=1)
y_test_multiclass = np.argmax(y_test, axis=1)

# Calculate the kappa score
kappa = cohen_kappa_score(y_test_multiclass, y_pred_multiclass)
print("The result of Kappa is :", round(kappa, 3))

# Generate the classification report
report = classification_report(y_test_multiclass, y_pred_multiclass, target_names= class_names)

# Print the report
print("The result of the classification report is: \n ",report)

In [ ]:
y_pred

Generate Confusion Matrix for inspection.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(
    y_test_multiclass,
    y_pred_multiclass)

# Create a ConfusionMatrixDisplay object
display = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=class_names)

# Create a figure with a larger size
fig = plt.figure(figsize=(11, 11))

# Create a subplot within the figure
ax = fig.subplots()

# Plot the confusion matrix as a heatmap
display.plot(ax=ax)

# Show the plot
plt.show()

Print sample predictions

In [ ]:
img_range = range(20)
imgs = x_test[img_range]
true_labels = [class_names[np.argmax(x)] for x in y_test[img_range]]
predictions = model.predict(imgs.reshape(len(img_range), img_rows, img_cols, channels))
pred_labels = [class_names[np.argmax(x)] for x in predictions]
titles = [pred_labels[x]+('' if true_labels[x] == pred_labels[x] else ' ('+true_labels[x]+')') for x in img_range]
plot_images(imgs, cols=5, figsize=(11,11), titles=titles)

Save the experiment results on to your computer. Repeat the above experiment with `model_2`. Compare the experiment results between the models. Was the preformance improved with `model_2`? and why?